In [1]:
import numpy as np
import pandas as pd
import unittest

In [2]:
class Laguerre:
    def __init__(self,beta,sigma):
        self.beta = beta
        self.sigma = sigma
            
    @property
    def beta(self):
        return self._beta
    
    @property
    def sigma(self):
        return self._sigma
    
    @beta.setter
    def beta(self, beta):
        self._beta = beta
        
    @sigma.setter
    def sigma(self, sigma):
        self._sigma = sigma
        
    def laguerre_polynom(self, t, n):
            
        if n == 0:
            return np.sqrt(self.sigma) * np.exp(-self.beta * t / 2)

        
        l_prev = np.sqrt(self.sigma) * (1 - self.sigma * t) * np.exp(-self.beta * t / 2)
        l_prev2 = np.sqrt(self.sigma) * np.exp(-self.beta * t / 2)

        for i in range(2, n + 1):
            temp = l_prev
            l_prev = (2 * i - 1 - self.sigma * t) * l_prev / i - (i - 1) * l_prev2 / i
            l_prev2 = temp

        return l_prev
    
    def laguerre_polynom_tabulation(self, n, t_max, t_step = 0.1):
        t_values = np.arange(0, t_max, t_step)

        laguerre_values = [self.laguerre_polynom(t=i,n=n) for i in t_values]

        result = pd.DataFrame(data={'t': t_values, f'L_{n}': laguerre_values})

        return result
    
    def laguerre_transform_tabulation(self, f, n_max, int_points = 10000):
        N = range(0, n_max+1)
        return pd.DataFrame({'n': N, f'L_n': [self.laguerre_transform(f=f, n_max=n) for n in N]})
    
    def inverse_laguerre_transform(self,h,t):
        return sum([h[k] * self.laguerre_polynom(t=t, n=k) for k in range(0, len(h))])

In [3]:
class Experiment:
    def __init__(self,laguerre):
        self.laguerre = laguerre
        
    @property
    def laguerre(self):
        return self._laguerre
    
    @laguerre.setter
    def laguerre(self, value):
        self._laguerre = value
        
    def experiment(self,n_max = 20, eps = 1e-3, t_max = 100, t_points = 1000):
        N = range(0, n_max + 1)
        suitable_t = 0

        for t in np.linspace(0, t_max, t_points):
            if all(abs(self.laguerre.laguerre_polynom(t, n)) <= eps for n in N):
                suitable_t = t
                break

        result = pd.DataFrame(data={'n': N, 'L_n':
                                    [self.laguerre.laguerre_polynom(suitable_t, n) for n in N]})

        return result, suitable_t

In [4]:
class Integral:
    def __init__(self, laguerre, experiment):
        self.laguerre = laguerre
        self.experiment = experiment

    def integral(self, f, a, b, int_points=1000):
        x = np.linspace(a, b, int_points)
        s = sum([f(i) for i in x])
        return s * abs(b - a) / int_points

    def laguerre_transform(self, f, n_max, int_points=10000):
        def integrand(t):
            alpha = self.laguerre.sigma - self.laguerre.beta
            return f(t) * self.laguerre.laguerre_polynom(t=t, n=n_max) * np.exp(-alpha * t)

        t_max = self.experiment.experiment(n_max=n_max)[0]

        return self.integral(integrand, 0, t_max, int_points)

In [5]:
class Laguerre_plot:
    def __init__(self, laguerre, integral):
        self.laguerre = laguerre
        self.integral = integral

    def laguerre_polynom_plot(self, t_max, n_max, t_step=0.01):
        plt.figure(figsize=(20, 15))

        for n in range(0, n_max + 1):
            l_n_tabulation = self.laguerre.laguerre_polynom_tabulation(n=n, t_max=t_max, t_step=t_step)
            plt.plot(l_n_tabulation['t'], l_n_tabulation[f'L_{n}'], label=f'L_{n}')

        plt.title('Laguerre Polynomials')
        plt.xlabel('t')
        plt.ylabel('L_n(t)')
        plt.legend()
        plt.grid()
        plt.show()

    def laguerre_transform_plot(self, f, n_max, t_max=np.pi * 2, t_step=0.01, int_points=10000):
        h = self.integral.laguerre_transform_tabulation(f=f, n_max=n_max)['L_n'].tolist()

        T = np.arange(0, t_max, t_step)
        inverse_laguerre_transform_tabulation = pd.DataFrame({'t': T, 'h': [self.integral.laguerre_transform(h=h, n_max=n_max, t=t) for t in T]})

        plt.figure(figsize=(10, 10))
        plt.plot(inverse_laguerre_transform_tabulation['t'], inverse_laguerre_transform_tabulation['h'])
        plt.title('Inverse Laguerre Transform')
        plt.xlabel('t')
        plt.ylabel('func(t)')
        plt.grid()
        plt.show()

In [6]:
class TestLaguerre(unittest.TestCase):
    def setUp(self):
        self.laguerre_instance = Laguerre(beta=2.0, sigma=4.0)

    def test_init(self):
        self.assertEqual(self.laguerre_instance.beta, 2.0)
        self.assertEqual(self.laguerre_instance.sigma, 4.0)
  

    def test_laguerre_polynom(self):
        laguerre = Laguerre(2, 4)
        self.assertEqual(laguerre.laguerre_polynom(3, 5), 2.728331346558944)
        self.assertEqual(laguerre.laguerre_polynom(0, 0), 2)
        self.assertEqual(laguerre.laguerre_polynom(1, 1), -2.207276647028654)
  
    def test_laguerre_polynom_tabulation(self):
        laguerre = Laguerre(2, 4)
        self.assertEqual(laguerre.laguerre_polynom_tabulation(10, 20).shape, (200, 2))
        self.assertAlmostEqual(laguerre.laguerre_polynom_tabulation(10, 20).iloc[0]['L_10'], 1.999999999999999)
        self.assertAlmostEqual(laguerre.laguerre_polynom_tabulation(10, 20).iloc[199]['L_10'], 3022.018782358826)   


In [7]:
class TestExperiment(unittest.TestCase):
    def test_init(self):
        laguerre = Laguerre(2, 4)
        experiment = Experiment(laguerre)
        self.assertEqual(experiment.laguerre, laguerre)
        
    def test_experiment(self):
        laguerre = Laguerre(beta=2, sigma=4)  
        exp = Experiment(laguerre)
        df, ans = exp.experiment(90)  
        self.assertEqual(df.shape, (91,2))  
        self.assertEqual(ans, 0) 
        self.assertEqual(df['L_n'][0], 2.0)
unittest.main(argv=[''], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.069s

OK
